In [1]:
import pandas as pd
import json
import re

In [8]:
#Cargamos el archivo json
df_memoria = pd.read_json('src/output_steam_games.json', lines=True)

In [9]:
df = df_memoria.copy()

In [11]:
df.tail()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,1.49,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,None,"Nikita ""Ghost_RUS""",None,NaN,None,NaN
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,4.24,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,None,Sacada,None,NaN,None,NaN
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,None,Laush Dmitriy Sergeevich,None,NaN,None,NaN
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,None,"xropi,stev3ns",None,NaN,None,NaN
120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,NaN,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None,None,None,NaN,None,NaN


In [16]:
#Limpiamos los datos

#Eliminamos los registros que parecen venir de la concatenación de otra tabla, usando la columna publisher como proxy para identificarlos
df = df.dropna(subset=['publisher'])

#Eliminamos las columnas que no nos interesan
df = df.drop(columns=['publisher', 'url', 'reviews_url', 'discount_price', 'early_access', 'developer', 'user_id', 'items', 'items_count', 'steam_id' ])

#Llevamos los nombres a minúsculas y quitamos caracteres especiales
df['app_name'] = df['app_name'].apply(lambda x: re.sub(r'[^\w\s]', '', x.lower()) if isinstance(x, str) else x)
df['title'] = df['title'].apply(lambda x: re.sub(r'[^\w\s]', '', x.lower()) if isinstance(x, str) else x)

In [25]:
df.loc[df['id'] == 211420]

,genres,app_name,title,tags,specs,price,id,metascore,steam_id
119097,"[Action, RPG]",dark souls prepare to die edition,dark souls prepare to die edition,"[RPG, Dark Fantasy, Difficult, Action, Explora...","[Single-player, Multi-player, Steam Achievemen...",19.99,211420.0,85,NaN
